## Import libaries

In [1]:
import os
import sys
assert os.path.exists('../src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements".


In [11]:
%load_ext autoreload
%autoreload 2


from src import preprocessor
from src.utils import load_documents
import nltk
import json
import ast
import pandas as pd
import time
import shutil
from threading import Thread

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from src.bulk_runner import OUTPUT_FOLDER
from src.bulk_runner import main_runner

2022-07-24 11:04:13.310175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-24 11:04:13.310198: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)

In [5]:
def delete_first_line(filename:str,done_file:str):
    with open(filename, 'r') as fin:
        data = fin.read().splitlines(True)
    with open(done_file, 'a+') as fout:
        fout.write(data[0])
    with open(filename, 'w') as fout:
        fout.writelines(data[1:])

In [6]:
combinations = []

for comb_filename in ['combinations.txt','combinations_ctm.txt']:
    with open(comb_filename) as comb_file:
        for i,item in enumerate(comb_file.readlines()):
            #item = item.replace("'",'"')
            x= ast.literal_eval(item)
            combinations.append((x,comb_filename))

In [12]:
timeout = 20 * 60
dfs_merged=pd.DataFrame()
for i,(configs,configs_filename) in enumerate(combinations):
    print(f'[INFO] Configs #{i}/{len(combinations)}: {configs}')
    try:
        delete_first_line(configs_filename,configs_filename + '_done')
        main_thread= Thread(target=main_runner, args=[configs])
        dfs_merged = main_thread.start()
        df_output_topic_word =  main_thread.join(timeout=timeout)
        bridge.set()
    # df_output_topic_word = main_runner(configs=configs)
    dfs_merged = pd.concat([dfs_merged,df_output_topic_word])
    dfs_merged.to_csv(f'./{OUTPUT_FOLDER}/merged.csv')
    except Exception:
        print('[WARN] Current execution gave an error!')
        time.sleep(10)


[INFO] Configs #0/1958: {'algorithm_args': {'embedding_model': 'universal-sentence-encoder', 'min_count': 50, 'algorithm': 'top2vec', 'num_topics': 10, 'doc2vec_speed': 'learn', 'umap_args': {'n_neighbors': 15, 'n_components': 3, 'metric': 'cosine', 'random_state': 42}, 'hdbscan_args': {'min_cluster_size': 5, 'metric': 'euclidean', 'cluster_selection_method': 'eom'}}, 'preprocessing_funcs': ['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_contractions', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_english_stop_words', 'lemmatize_noun'], 'dataset': 'crisis_17'}
[INFO] [PREPROCESSOR] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_new_lines', 'remove_extra_spaces', 'remove_html_tags', 'expand_contractions', 'remove_english_stop_words', 'lemmatize', 'lemmatize_verb', 'lemmatize_noun', 'lemmat

2022-07-24 11:06:25,159 - top2vec - INFO - Pre-processing documents for training
2022-07-24 11:06:25,159 INFO:Pre-processing documents for training


[INFO] [PREPROCESSOR] Preprocessing completed in 7.258 seconds..
[INFO] Running with 10 topics.
[INFO] Top2Vec Parameters:
{
    "embedding_model": "universal-sentence-encoder",
    "min_count": 50,
    "algorithm": "top2vec",
    "num_topics": 10,
    "doc2vec_speed": "learn",
    "umap_args": {
        "n_neighbors": 15,
        "n_components": 3,
        "metric": "cosine",
        "random_state": 42
    },
    "hdbscan_args": {
        "min_cluster_size": 5,
        "metric": "euclidean",
        "cluster_selection_method": "eom"
    },
    "data_name": "crisis_17",
    "run_id": 1658653577
}
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_distiluse-base-multilingual-cased" found, so no need to download.
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_all-MiniLM-L6-v2" found, so no need to download.
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2" found, so n

2022-07-24 11:06:27,767 - top2vec - INFO - Loading universal-sentence-encoder model at ./pretrained_models/universal-sentence-encoder
2022-07-24 11:06:27,767 INFO:Loading universal-sentence-encoder model at ./pretrained_models/universal-sentence-encoder
2022-07-24 11:06:27.900914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-24 11:06:27.900938: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-24 11:06:27.900955: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (iceking-ThinkPad-T490): /proc/driver/nvidia/version does not exist
2022-07-24 11:06:27.901110: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CP

KeyboardInterrupt: 